In [ ]:
!pip install python-dotenv
!pip install langchain
!pip install langchain-openai

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OpenAi Key"]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [ ]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

class Analyst(BaseModel):
    affiliation: str = Field(
        description= "Primary affiliation of the analyst",
    )
    name: str = Field(
        description="Name of the analyst"
    )
    role: str = Field(
        description="Role of the analyst in the context of the topic",
    )
    description: str = Field(
        description = "Description of the analyst focusm concerns, and motives",

    )
    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"
    
class Perspectives(BaseModel):
    analysts: List[Analyst] = Field(
        description="Comprehensive list of analysts with their roles and affiliations",
    )
class GenerateAnalystsState(TypedDict):
    topic: str
    max_analysts: int
    human_analysts_feedback: str
    analysts: List[Analyst]

    


BUILDING THE AI ANALYST SUB GRAPH....   

In [ ]:
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

analyst_instructions = """You are tasked with creating a set of AI analysts personas. Follow these
instructions carefully 'dusthead': 

1. First, review the research topic:
{topic}

2. Examine any editorial feedback that has been optimally provided to guide creation of the analysts:
{human_analyst_feedback}

3. Determine the most interesting themes based upon documents and / or feedback above.

4. Pick the top {max_analysts} themes.

5. Assign one anlyst to each theme.

"""

def create_analysts(state: GenerateAnalystsState):

    """Create Analysts..."""

    topic= state['topic']
    max_analysts=state["max_analysts"]
    human_analysts_feedback= state.get("human_analysts_feedback", "")

    structured_llm = llm.with_structured_output(Perspectives)

    system_message = analyst_instructions.format(topic=topic,
                                                 human_analysts_feedback=human_analysts_feedback,
                                                 max_analysts=max_analysts)
    analysts = structured_llm.invoke([SystemMessage(content=system_message)] + [HumanMessage(content="Generate the set of analysts")])

    return {"analysts": analysts.analysts}

def human_feedback(state:GenerateAnalystState):
    """No-op node that should be interrupted on """
    pass

def should_continue(state:GenerateAnalystsState):
    """Return the next node to execute"""

    human_analyst_feedback = state.get("human_analyst_feedback", None)

    if human_analyst_feedback:
        return "create_analysts"
    

    return END

builder = StateGraph(GenerateAnalystsState)

builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)

builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", should_continue, ["create_analysts", END])

memory = MemorySaver()
graph = builder.compile(interrupt_before=["human_feedback"], checkpointer=memory)

display(Image(graph.get_graph(xray=1).draw_mermaid_png()))



RUNNING SUB GRAPH WITH EXAMPLE TO SEE IF IT WORKS....

In [ ]:
max_analysts = 3
topic= "The benefits of adopting LangGraph as an agent framework"
thread = {"configurable": {"thread_id": "1"}}

for event in graph.stream({"topic":topic, "max_analysts": max,}, thread, stream_mode="values"):

    analysts = event.get("analysts", "")

    if analysts:
        for anlyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-"* 50)

            




HUMAN FEEDBACK EXAMPLE....

In [ ]:
## This code is getting the current state of the node....
state= graph.get_state(thread)
state.next

HERE I WILL ALLOW "HUMAN FEEDBACK". IN CASE THE USER DOES NOT LIKE THE RESULTS GENERATED....

In [ ]:
graph.update_state(thread, {"human_analyst_feedback":
                            "Add in someone from a startup to add an entrepreneur of sorts."}, as_node="human_feedback")

FEEDBACK GIVEN. LETS NOW CONTINUE WITH THE EXECUTION OF THE APPLICATION....

In [ ]:
for event in graph.stream(None, thread, stream_mode="values"):

    analysts= event.get("analysts", "")
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 50)

            

If I am satisfied with the new analysts provided. I will provide no additional human feedback so the app can continue the execution with 3 of the analysts. To do that, I set "further_feedback" to "None".

In [ ]:
further_feedback = None
graph.update_state(thread, {"human_analyst_feedback":
                    further_feedback}, as_node="human_feedback")

In [ ]:
## No more human feedback. Continue excution of agent flow....

for event in graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name= next(iter(event.keys()))
    print(node_name)

In [ ]:
final_state = graph.get_state(thread)
analysts = final_state.values.get("analysts")



In [ ]:
final_state.next

In [ ]:
for analyst in analysts:
    print(f"Name: {analyst.name}")
    print(f"Affiliation: {analyst.affiliation}")
    print(f"Role: {analyst.role}")
    print(f"Description: {analyst.desription}")
    print("-" * 50)

SECOND SUB GRAPH THE "INTERVIEW GENERATOR"  SUB GRAPH....

In [ ]:
import operator
from typing import Annotated
from langgraph.graph import MessagesState

class InterviewState(MessagesState):
    max_num_turns = int
    context: Annotated[List, operator.add]
    analyst: Analyst
    interview: str
    sections: list

class SearchQuery(BaseModel):
    search_query: str = False(None, description= "Search query for retrieval.")


    



Define the noDe generate_question....

In [ ]:
question_instructions = """You are an analyst tasked with interviewing an expert to learn about a specific topic. 

Your goal is boil down to interesting and specific insights related to your topic.

1. Interesting: Insights that people will find surprising or non-obvious.
        
2. Specific: Insights that avoid generalities and include specific examples from the expert.

Here is your topic of focus and set of goals: {goals}
        
Begin by introducing yourself using a name that fits your persona, and then ask your question.

Continue to ask questions to drill down and refine your understanding of the topic.
        
When you are satisfied with your understanding, complete the interview with: "Thank you so much for your help!"

Remember to stay in character throughout your response, reflecting the persona and goals provided to you."""


def generate_questions(state: InterviewState):
    """Node to generate a question"""

    ## get the state....
    analyst = state["analyst"]
    messages = state["messages"]


    ## generate questions...

    system_message = question_instructions.format(goals=analyst.persona)
    question = llm.invoke([SystemMessage(content=system_message)]+messages)

    ## Write messages to state

    return {"messages": [question]}






